In [1]:
import numpy as np
import pandas as pd

In [2]:
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [3]:
Corpus = pd.read_csv("train.csv")

In [4]:
Corpus.shape
Corpus.info()
Corpus.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
Corpus_WO_Label = Corpus.drop('label', axis=1)
Corpus_Label = Corpus['label']

In [6]:
Corpus_Label.value_counts()

1    10413
0    10387
Name: label, dtype: int64

In [7]:
print(Corpus['text'])

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        Ever get the feeling your life circles the rou...
2        Why the Truth Might Get You Fired October 29, ...
3        Videos 15 Civilians Killed In Single US Airstr...
4        Print \nAn Iranian woman has been sentenced to...
                               ...                        
20795    Rapper T. I. unloaded on black celebrities who...
20796    When the Green Bay Packers lost to the Washing...
20797    The Macy’s of today grew from the union of sev...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799      David Swanson is an author, activist, journa...
Name: text, Length: 20800, dtype: object


In [11]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
# Pandas function to remove blank rows , if inplace is set to true, dataframe is updated without returning any value
Corpus['text'].dropna(inplace=True)
# Convert text to lowercase since python is case senstive
Corpus['text'] = [str(text).lower() for text in Corpus['text']]
# In this each text in Corpus will be broken into set of words
Corpus['text']= [word_tokenize(str(text)) for text in Corpus['text']]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
# Creating a dictionary of pos_tags to understand the type of word whether it is noun / adjective / adverb / verb. 
# Default value of pos_tags is noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['R'] = wn.ADV
tag_map['V'] = wn.VERB
tag_map['J'] = wn.ADJ

In [ ]:
for index,text in enumerate(Corpus['text']):
    final_words = []
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function provides whether the word is Noun/Verb/Adverb/Adjective or something else.
    for word, tag in pos_tag(text):
        # To remove stop words and consider only alphabetical words
        if word not in stopwords.words('english') and word.isalpha():
            final_word = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            final_words.append(final_word)
    # After each iteration, final_words will be stored in corpus
    Corpus.loc[index,'text_final'] = str(final_words)

In [ ]:
Train_Corpus_WO_Label, Val_Test_Corpus_WO_Label, Train_Label, Val_Test_Label = model_selection.train_test_split(Corpus['text_final'],Corpus['label'],test_size=0.4)
Validation_Corpus_WO_Label, Test_Corpus_WO_Label, Validation_Label, Test_Label = model_selection.train_test_split(Val_Test_Corpus_WO_Label,Val_Test_Label,test_size=0.5)

In [ ]:
Tfidf_vector = TfidfVectorizer(max_features=6000)
Tfidf_vector.fit(Corpus['text_final'])
Train_Corpus_WO_Label_Tfidf = Tfidf_vector.transform(Train_Corpus_WO_Label)
Validation_Corpus_WO_Label_Tfidf = Tfidf_vector.transform(Validation_Corpus_WO_Label)

In [ ]:
print(Tfidf_vect.vocabulary_)

In [ ]:
nb = naive_bayes.MultinomialNB()
nb.fit(Train_Corpus_WO_Label_Tfidf,Train_Label)
# Validation dataset used for predicting labels
predictions_NB = nb.predict(Validation_Corpus_WO_Label_Tfidf)
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Validation_Label)*100)

In [ ]:
#SVM Classfier
# 
SVM = svm.SVC(C=1.0, kernel='linear', gamma='auto')
SVM.fit(Train_Corpus_WO_Label_Tfidf,Train_Label)
# Validation dataset used for predicting labels
predictions_SVM = SVM.predict(Validation_Corpus_WO_Label_Tfidf)
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Validation_Label)*100)

In [ ]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

Logistic_Reg = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000)
model=Logistic_Reg.fit(Train_Corpus_WO_Label_Tfidf,Train_Label)
predictions_Logreg = Logistic_Reg.predict(Validation_Corpus_WO_Label_Tfidf)
print("Logistic Regression Accuracy Score -> ",accuracy_score(predictions_Logreg, Validation_Label)*100)

In [ ]:
Test_Corpus_WO_Label_Tfidf = Tfidf_vector.transform(Test_Corpus_WO_Label)

In [ ]:
# Test dataset used for predicting labels
test_predictions_NB = nb.predict(Test_Corpus_WO_Label_Tfidf)
print("Naive Bayes Accuracy Score -> ",accuracy_score(test_predictions_NB, Test_Label)*100)

In [ ]:
test_predictions_SVM = SVM.predict(Test_Corpus_WO_Label_Tfidf)
print("SVM Accuracy Score -> ",accuracy_score(test_predictions_SVM, Test_Label)*100)

In [ ]:
test_predictions_Logreg = Logistic_Reg.predict(Test_Corpus_WO_Label_Tfidf)
print("Logistic Regression Accuracy Score -> ",accuracy_score(test_predictions_Logreg, Test_Label)*100)